In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
#| echo: false
import pandas as pd
import os
import numpy as np
import redcap
from IPython.display import Markdown, display
import papermill as pm

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\langhe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\langhe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
#| tags: [parameters]
root    = r"C:\Users\langhe\switchdrive\Private\Collaborations\2023 EDCPT ultrasound\WP3\dictionaries"
in_csv1 = os.path.join(root, r"IMCIPLUSRCTPilot_DataDictionary_2024-06-07.csv")
in_csv2 = os.path.join(root, r"IMCIPLUSRCTPilot_DataDictionary_2024-06-10.csv")

In [5]:
d1 = redcap.DataDic(in_csv1,
                    dtype = "RCT")
d2 = redcap.DataDic(in_csv2,
                    dtype = "RCT")

# Characteristics of the REDCap data dictionary

In [6]:
s1 = d2.printNumForms()

In [7]:
display(Markdown(f'{s1}'))

The REDCap data dictionary contains 14 forms.

In [8]:
#| label: tbl-forms
#| tbl-cap: "Number of variables by forms"
d2.getForms()

,Forms,N
0,a_enrollment,4
1,a_exit_interview,66
2,a_phone_followup_d8,20
3,admitted_in_ward_follow_up,26
4,b1_screening_registration,19
5,b2_screening_antibiotic_use,3
6,b3_screening_vitalsigns,53
7,b_day_8_inperson_followup,34
8,b_patient_information,15
9,b_spontaneous_follow_up,18


In [9]:
q2 = d2.getVariables()

In [10]:
#| tbl-cap: "Extract of the data dictionary variables (first 10 variables)"
q2[["Variables", "Forms", "Field Type", "Field Label"]].head(10)

,Variables,Forms,Field Type,Field Label
0,record_id,b1_screening_registration,text,Screening /Study ID
1,screening_date,b1_screening_registration,text,Screening date
2,q2_prev_enrolled,b1_screening_registration,radio,Was the child previously enrolled in this stud...
3,q2_b_enrollment_date,b1_screening_registration,yesno,Is date of enrolment known ?
4,q2_a,b1_screening_registration,text,Date of enrolment
5,q2i_unknown_date,b1_screening_registration,radio,"If unknown, was enrolment >28days ago"
6,scr_prevenrollment28d,b1_screening_registration,calc,Enrolment into IMCI-PLUS study in the past 28...
7,please_exit_screening_and,b1_screening_registration,descriptive,"<div class=""rich-text-field-label""><p><span st..."
8,child_dob,b1_screening_registration,text,Child DoB
9,unknown_dob,b1_screening_registration,yesno,Are you sure the date of birth is unknown?


In [11]:
#| tbl-cap: "List of personally identifiable information variables"
d2.getIdentifiers()

,Variables,Forms,Field Type,Field Label
81,l_name,b_patient_information,text,Last name:
82,f_name,b_patient_information,text,First Name
85,id_num,b_patient_information,text,Patient's National ID number
86,caregiver_name,b_patient_information,text,Primary Caregiver's Last name:
87,caregiver_surname,b_patient_information,text,Primary Caregiver's First name:
88,alt_contact_person,b_patient_information,text,Alternate Contact person (Name & Surname):
89,alt_contact_num,b_patient_information,text,Alternate Contact person (Cell phone number)
90,legal_rep,b_patient_information,text,Legal representative of child in the event of ...
91,legal_cell,b_patient_information,text,Cell phone number of the legal representative\...


# Comparison with a reference REDCap data dictionary

In [12]:
d1 = redcap.DataDic(in_csv1,
                    dtype = "RCT")

In [13]:
s2 = d2.compareNumForms(d1)

In [14]:
display(Markdown(f'{s2}'))

Increased number of forms: 14 vs. 11

In [15]:
_, identical_df, samename_df, renamed_df, added_df = d2.detectIdenticalVariables(d1)

## Same forms

In [16]:
same_forms_df = d2.detectModificationsInSameForms(d1)

In [17]:
#| tbl-cap: "Overview of modifications in same forms"
same_forms_df

,Forms,N,N0,No changes,Changes
0,admitted_in_ward_follow_up,26,31,8,5 Variables deleted
1,phone_followup_d29,13,13,13,Same number of variables


In [18]:
form_list = same_forms_df["Forms"].unique().tolist()
cols = [i for i in q2.columns.tolist() if i not in ["Forms", "index"]]

In [19]:
identicals = dict()
samenames = dict()
renamed = dict()
for k in form_list:
    identicals[k] = identical_df[identical_df["Forms"] == k]
    samenames[k] = samename_df[samename_df["Forms"] == k]
    renamed[k] = renamed_df[renamed_df["Forms"] == k]

### Identical variables

In [20]:
# Display results dynamically
for k, v in identicals.items():
    display(Markdown(f'#### Form: {k}'))
    display(Markdown(f'{len(v.index)} variables'))
    v = v[["Variables", "Field Type", "Field Label", "Choices, Calculations, OR Slider Labels"]]
    display(v)

#### Form: admitted_in_ward_follow_up

8 variables

,Variables,Field Type,Field Label,"Choices, Calculations, OR Slider Labels"
101,convulsions,radio,Convulsions with this illness,"1, Yes | 0, No"
149,desc_findings,descriptive,"<div class=""rich-text-field-label""><p><span st...",NaN
211,follow_up_date,text,Date,NaN
214,follow_up_desc,descriptive,"<div class=""rich-text-field-label""><p><span st...",NaN
228,general_status,radio,General Status of Patient,"1, Improving | 2, Unchanged | 3, Worsening"
529,stridor_present,yesno,Stridor present,NaN
556,vomiting,radio,Vomiting everything,"1, Yes | 0, No"
577,working_diagnosis,text,Working Diagnoses:,NaN


#### Form: phone_followup_d29

13 variables

,Variables,Field Type,Field Label,"Choices, Calculations, OR Slider Labels"
6,admission_date_dd3afa,text,When admitted/re-admitted:,NaN
108,d8_date_155715,text,Date,NaN
114,d8_no_reason_c9c481,text,If No: Reason,NaN
119,d8_phone_call_bc9be7,yesno,Did the D8 phone call visit F/up take place:,NaN
131,death_cause_63ef7e,text,Suspected cause of death:,NaN
134,death_date_eca754,text,"if death, when:",NaN
160,discharged_date_b507cc,text,Discharged Date,NaN
240,hospi_related_illness,yesno,Hospitalization related to initial illness?,NaN
249,hospitalization_4d4ecb,radio,"Hospitalization (Defined as ""Slept overnight i...","1, Not Admitted | 2, Admitted on D1 (on day of..."
254,hospitalization_place_278cb2,text,Where:,NaN


### Modified variables (name unchanged)

In [21]:
# Display results dynamically
for k, v in samenames.items():
    display(Markdown(f'#### Form: {k}'))
    display(Markdown(f'{len(v.index)} variables'))
    v = v[cols].dropna(axis=1, how="all")
    display(v)

#### Form: admitted_in_ward_follow_up

11 variables

,Variables,Section Header,Field Type,Field Label,"Choices, Calculations, OR Slider Labels",Text Validation Type OR Show Slider Number,Text Validation Min,Text Validation Max,Branching Logic (Show field only if...),Required Field?,Matrix Group Name
56,day,NaN,text,Study Day,NaN,integer,1.0,28,NaN,y,NaN
80,drink_inability,"If yes, which ones:",radio,Inability to drink/breastfeed (assessed by pro...,"1, Yes | 0, No",NaN,NaN,NaN,[who_danger_signs] = '1',y,who_imci_danger_sign_matrix
133,high_care_reason,NaN,text,Reason for hospital transfer:,NaN,NaN,NaN,NaN,[level_of_care_change] = '3',y,NaN
137,hospital_name,NaN,text,Name of hospital to which patient was referred...,NaN,NaN,NaN,NaN,[level_of_care_change] = '3',y,NaN
157,lethargy,NaN,radio,Lethargy/unconscious (assessed by provider),"1, Yes | 0, No",NaN,NaN,NaN,[who_danger_signs] = '1',y,who_imci_danger_sign_matrix
160,level_of_care_change,NaN,radio,Was the patient's level of care changed since ...,"0, no | 1, admission to high care unit | 2, ad...",NaN,NaN,NaN,NaN,y,NaN
210,patient_discharged,NaN,yesno,Patient discharged home:,NaN,NaN,NaN,NaN,NaN,y,NaN
264,respi_need_support,NaN,radio,Is patient currently receiving respiratory sup...,"0, none | 1, HFNC | 2, bubble CPAP | 3, CPAP |...",NaN,NaN,NaN,NaN,y,NaN
266,respiratory_distress,NaN,radio,Severe respiratory distress (please provide ov...,"0, no | 1, new since last study encounter | 2,...",NaN,NaN,NaN,NaN,y,NaN
306,wheeze_present,NaN,checkbox,Clinical signs of bronchiolitis/ reactive airw...,"0, no | 1, audible wheezing | 2, wheezing (aus...",NaN,NaN,NaN,NaN,y,NaN


#### Form: phone_followup_d29

0 variables

""


### Renamed variables

In [22]:
# Display results dynamically
cols1 = cols.append("Variables0")
cols1

## Renamed forms

In [23]:
renamed_forms_df = d2.detectModificationsInRenamedForms(d1)

In [24]:
#| tbl-cap: "Overview of modifications in renamed forms"
renamed_forms_df

,Forms,N,Forms0,N0,No changes,Changes
0,a_enrollment,4,enrollment,3,0,1 Variables added
1,a_phone_followup_d8,20,phone_followup_d8,20,0,Same number of variables
2,b1_screening_registration,19,screening_registration,14,0,5 Variables added
3,b2_screening_antibiotic_use,3,screening_antibiotic_use,2,0,1 Variables added
4,b3_screening_vitalsigns,53,screening_vitalsigns,48,0,5 Variables added
5,b_day_8_inperson_followup,34,day_8_inperson_followup,30,0,4 Variables added
6,b_patient_information,15,patient_information,15,0,Same number of variables
7,c_lung_ultrasound,31,lung_ultrasound,27,0,4 Variables added


In [25]:
form_list = renamed_forms_df["Forms"].unique().tolist()
cols = [i for i in q2.columns.tolist() if i != "Forms"]
outs = dict()
for k in form_list:
    outs[k] = q2[q2["Forms"] == k][cols]
# Display results dynamically
for k, v in outs.items():
    display(Markdown(f'### Form: {k}'))
    v = v.dropna(axis=1, how="all")
    display(v)

### Form: a_enrollment

,index,Variables,Section Header,Field Type,Field Label,Field Note,Branching Logic (Show field only if...),Required Field?
75,75,consent_rct,"<div class=""rich-text-field-label""><div class=...",yesno,Did the caregiver sign the informed consent fo...,if No - [END FORM],[eligibility_rct] = '1',y
76,76,consent_observational,NaN,yesno,Did the caregiver sign the informed consent fo...,if No - [END FORM],[eligibility_observational] = '1',y
77,77,consent_biobanking,NaN,yesno,Did the caregiver sign the informed consent fo...,if No - [END FORM],[eligibility_general] = '1',y
78,78,please_randomize_patient,NaN,descriptive,"<div class=""rich-text-field-label""><p><span st...",NaN,[consent_rct] = '1',NaN


### Form: a_phone_followup_d8

,index,Variables,Section Header,Field Type,Field Label,"Choices, Calculations, OR Slider Labels",Text Validation Type OR Show Slider Number,Branching Logic (Show field only if...),Required Field?
266,266,d8_date,NaN,text,Date,NaN,date_dmy,NaN,y
267,267,d8_phone_call,NaN,yesno,Did the D8 phone call visit F/up take place:,NaN,NaN,NaN,y
268,268,d8_no_reason,NaN,text,If No: Reason,NaN,NaN,[d8_phone_call] = '0',y
269,269,patient_outcome,NaN,radio,Patient Outcome:,"1, Resolved Completely (Cured) | 2, Improved |...",NaN,NaN,y
270,270,death_date,NaN,text,"if so, when",NaN,date_dmy,[patient_outcome] = '5',y
271,271,death_cause,NaN,text,Suspected cause:,NaN,NaN,[patient_outcome] = '5',y
272,272,attened_clinic,Further health seeking,yesno,Attended a clinic/GP/OPD/ER,NaN,NaN,NaN,y
273,273,date_attended_clinic,NaN,yesno,Date attended a clinic/GP/OPD/ER,NaN,NaN,[attened_clinic] = '1',y
274,274,medication_prescribed,NaN,notes,Clinic/OPD/ER Medication / treatment prescribed:,NaN,NaN,[attened_clinic] = '1',y
275,275,gp_antibiotic,NaN,text,What: Antibiotic,NaN,NaN,[attened_clinic] = '1',y


### Form: b1_screening_registration

...

,index,Variables,Section Header,Field Type,Field Label,"Choices, Calculations, OR Slider Labels",Field Note,Text Validation Type OR Show Slider Number,Branching Logic (Show field only if...),Required Field?,Matrix Group Name,Field Annotation
286,286,actual_follow_up_date,NaN,text,Date,NaN,NaN,date_dmy,NaN,y,NaN,NaN
287,287,d8_phone_call_7ed0e4,NaN,yesno,Did the D8 in-person visit f/up take place:,NaN,NaN,NaN,NaN,y,NaN,NaN
288,288,d8_no_reason_c17839,NaN,text,If No: Reason,NaN,NaN,NaN,[d8_phone_call_7ed0e4] = '0',y,NaN,NaN
289,289,desc_caregiver_history,NaN,descriptive,"<div class=""rich-text-field-label""><p><span st...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
290,290,patient_outcome_510e6b,NaN,radio,Patient Outcome (According to caregiver):,"1, Resolved Completely (Cured) | 2, Improved |...",NaN,NaN,NaN,y,NaN,NaN
291,291,in_person_visit,Further health seeking,yesno,Attended today due to study staff phone call a...,NaN,NaN,NaN,NaN,y,NaN,NaN
292,292,attened_clinic_ea407f,NaN,radio,Between initial encounter and today: attended ...,"1, yes | 0, no | 98, unknown",NaN,NaN,NaN,y,NaN,NaN
293,293,fu_addvisit_studyd8,Additional health seeking encounters between i...,radio,Today due to study staff phone call advising a...,"1, yes | 2, no | 98, unknown",NaN,NaN,NaN,NaN,fu_addvisit,NaN
294,294,fu_addvisit_study,NaN,radio,Spontaneous follow-up with study team,"1, yes | 2, no | 98, unknown",NaN,NaN,NaN,NaN,fu_addvisit,NaN
295,295,fu_addvisit_opd,NaN,radio,"Outpatient provider (clinic, GP, ED...)","1, yes | 2, no | 98, unknown",NaN,NaN,NaN,NaN,fu_addvisit,NaN


### Form: b_patient_information

,index,Variables,Field Type,Field Label,"Choices, Calculations, OR Slider Labels",Text Validation Type OR Show Slider Number,Identifier?,Branching Logic (Show field only if...),Required Field?
79,79,p_folder_num,text,Patient's Folder Number:,NaN,NaN,NaN,NaN,y
80,80,p_folder_confirm,text,Patient's Folder Number confirmation:,NaN,NaN,NaN,NaN,NaN
81,81,l_name,text,Last name:,NaN,NaN,y,NaN,y
82,82,f_name,text,First Name,NaN,NaN,y,NaN,y
83,83,sex,radio,Sex,"1, Male | 2, Female",NaN,NaN,NaN,y
84,84,dob_p,text,DOB:,NaN,date_dmy,NaN,NaN,y
85,85,id_num,text,Patient's National ID number,NaN,identification_za,y,NaN,y
86,86,caregiver_name,text,Primary Caregiver's Last name:,NaN,NaN,y,NaN,NaN
87,87,caregiver_surname,text,Primary Caregiver's First name:,NaN,NaN,y,NaN,NaN
88,88,alt_contact_person,text,Alternate Contact person (Name & Surname):,NaN,NaN,y,NaN,NaN


### Form: c_lung_ultrasound

,index,Variables,Section Header,Field Type,Field Label,"Choices, Calculations, OR Slider Labels",Text Validation Type OR Show Slider Number,Branching Logic (Show field only if...),Required Field?,Matrix Group Name
94,94,ultrasound,NaN,yesno,Lung Ultrasound Collected:,NaN,NaN,NaN,y,NaN
95,95,exit_form,NaN,descriptive,"<div class=""rich-text-field-label""><p><span st...",NaN,NaN,[ultrasound] = '0',NaN,NaN
96,96,lus_nolusreasonyn,NaN,checkbox,Reason why LUS was not collected:,"1, patient refused | 2, patient deteriorated c...",NaN,[ultrasound] = '0',NaN,NaN
97,97,lus_nolusreasonyn_2,NaN,text,Other reason why LUS was not collected:,NaN,NaN,[lus_nolusreasonyn(3)] = '1',NaN,NaN
98,98,lus_id,NaN,text,LUS identifier on device,NaN,NaN,NaN,NaN,NaN
99,99,lus_date,NaN,text,When was this LUS performed:,NaN,date_dmy,NaN,NaN,NaN
100,100,lus_visittype,NaN,radio,Type of visit for which LUS was performed,"1, D1 | 2, D2-7 admitted patient | 3, D2-7 spo...",NaN,NaN,NaN,NaN
101,101,patience_postition,NaN,radio,Patient Position:,"1, Sitting | 2, Lying Down",NaN,NaN,NaN,NaN
102,102,exam_problem,NaN,yesno,Problem during exam?,NaN,NaN,NaN,NaN,NaN
103,103,lus_problem_type,NaN,checkbox,"If yes, what kind?:","1, patient crying | 2, patient moved a lot | 3...",NaN,[exam_problem] = '1',NaN,NaN


## New forms

New forms did not exist in the reference version of the REDCap data dictionary.

In [26]:
#| tbl-cap: "Overview of modifications in new forms"
d2.detectModificationsInRenamedForms(d1)

,Forms,N,Forms0,N0,No changes,Changes
0,a_enrollment,4,enrollment,3,0,1 Variables added
1,a_phone_followup_d8,20,phone_followup_d8,20,0,Same number of variables
2,b1_screening_registration,19,screening_registration,14,0,5 Variables added
3,b2_screening_antibiotic_use,3,screening_antibiotic_use,2,0,1 Variables added
4,b3_screening_vitalsigns,53,screening_vitalsigns,48,0,5 Variables added
5,b_day_8_inperson_followup,34,day_8_inperson_followup,30,0,4 Variables added
6,b_patient_information,15,patient_information,15,0,Same number of variables
7,c_lung_ultrasound,31,lung_ultrasound,27,0,4 Variables added


## Deleted forms

Deleted forms existed in the reference version of the REDCap data dictionary, but no longer exist.